In [10]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(lubridate)

In [11]:
### Reading in Data
fp <- '/pool001/mfzhao/PROCESSED_DATA/'
ecad_stations <- read_csv(str_c(fp, "ecad_stations.csv"),
                          col_types = cols(
                              STAID = col_character(),
                              STANAME = col_character(),
                              lat = col_double(),
                              lng = col_double()
                 ))
ghcn_stations <- read_csv(str_c(fp, "ghcn_stations.csv"))

ecad <- read_csv(str_c(fp, "ecad_weather.csv"), 
                 col_types = cols(
                     STAID = col_character(),
                     DATE = col_double(),
                     PRCP = col_double(),
                     TMAX = col_double()
                 ))
ghcn <- read_csv(str_c(fp, "ghcn_weather.csv"))

Parsed with column specification:
cols(
  STAID = col_character(),
  STANAME = col_character(),
  CN = col_character(),
  lat = col_double(),
  lng = col_double()
)

Parsed with column specification:
cols(
  STAID = col_character(),
  DATE = col_double(),
  PRCP = col_double(),
  TMAX = col_double()
)



In [12]:
CN <- read_csv(str_c(fp, "eu_mobility.csv")) %>%
    mutate(CN = str_sub(key, 1, 2)) %>%
    select(CN) %>%
    distinct()

CN <- data.frame(CN = c(CN$CN, 'US'), stringsAsFactors = F)

Parsed with column specification:
cols(
  key = col_character(),
  ds = col_date(format = ""),
  btvrc = col_double(),
  rstu = col_double(),
  n = col_double()
)



In [13]:
ecad_stations %>%
    inner_join(CN) %>%
    inner_join(ecad %>% select(STAID) %>% distinct()) -> ecad_stations

ghcn_stations %>%
    inner_join(CN) %>%
    inner_join(ghcn %>% select(STAID) %>% distinct()) -> ghcn_stations

Joining, by = "CN"

Joining, by = "STAID"

Joining, by = "CN"

Joining, by = "STAID"



In [14]:
ecad_stations %>% 
    select(STANAME, CN) %>%
    inner_join(ghcn_stations %>% 
               select(STANAME, CN) %>%
               distinct()) %>%
    distinct() -> intersection

ghcn_stations %>% 
    anti_join(intersection) -> ghcn_stations

ecad_stations %>% 
    mutate(STAID = str_c('ECAD', str_pad(STAID, 5, pad = '0'))) %>%
    bind_rows(ghcn_stations) -> stations

Joining, by = c("STANAME", "CN")

Joining, by = c("STANAME", "CN")



In [15]:
ecad %>% 
    mutate(STAID = str_c('ECAD', str_pad(STAID, 5, pad = '0'))) %>%
    bind_rows(ghcn %>%
              semi_join(ghcn_stations)) -> weather

weather %>%
    mutate(DATE = ymd(DATE),
           PRCP = PRCP/10,
           TMAX = TMAX/10) %>% 
    filter(DATE < as.Date('2020-07-01')) -> weather 

Joining, by = "STAID"



In [16]:
write_csv(stations, str_c(fp, 'stations.csv'))
write_csv(weather, str_c(fp, 'weather.csv'))